<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

This part of the project scrapes data from a wikipedia page containing postal codes in Toronto, CA.  It places the data in a pandas dataframe. 

As a first order of business, I will download various needed libraries.  This notebook was developed locally using Anaconda 3 with a virtual environment containing the needed libraries.  If they are not present (conda-forge geopy, folium) then uncomment the #!conda install lines.  Some of these libraries are not used in the first part of the project, but are loaded now by convention.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download Dataset

Here, the BeautifulSoup library is downloaded:

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import lxml
import html5lib

This is the target webpage:

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
type(soup)


bs4.BeautifulSoup

This code extracts the HTML table element from the page and stores it as My_table

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})

Create a new Pandas DataFrame for our data

In [5]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
postdf=pd.DataFrame(columns=column_names)

Ignoring the first row (containing column names) each row is read, cleaned, and, if appropriate, appended to the previously created DataFrame.

In [6]:
firstRow = True
for myrow in My_table.find_all('tr'):   # tr is HTML for Table Row
    if firstRow == False:
        lst=myrow.text.split('\n')  # Each row reads in as a blank, three data items, and another blank
        pc=lst[1]                   # PostalCode
        b=lst[2]                    # Borough
        neigh=lst[3]                # Neighborhoon
        if neigh == 'Not assigned': # Unassigned neighborhoods are assigned to the Borough name
            neigh = b
        rowdf = pd.DataFrame([[pc, b, neigh]], columns=column_names)
        if b != 'Not assigned':     # if Borough is unassigned, ignore the row
            postdf = postdf.append(rowdf, ignore_index=True)
    else:
        firstRow = False

print(f"DataFrame contains {postdf.shape[0]} rows.")

DataFrame contains 210 rows.


Written and submitted by Tom Kenny for IBM Data Science Capstone Course on Coursera